In [1]:
import numpy as np

#### Common and antithetic random variables 

Let $X$ and $Y$ be random variables with known cdfs, F and G, respectively. Suppose that we need to estimate $l = E[X - Y]$ via simulation. The simplest unbiased estimator of $l$ is $X - Y$. <br>
Suppose that $H_{1}$ and $H_{2}$ are real-values monotone functions. The problem can be formulated as follows: <br><br>
Within the set of all two-dimensional joint cdfs of (X, Y), find a joint cdf, $F^{*}$, that minimizes $Var(H_{1}(X) - H_{2}(Y))$, subject to X and Y having the prescribed F and G, respectively. <br><br>

X and Y need not be independent: <br><br>
$Var(X - Y) = Var(X) + Var(Y) - 2Cov(X, Y)$

As X and Y have determined variance (specified by F and G), we can try to minimize the variance by maximizing correlation between variables. <br>
 
Inverse transform method gives us: <br>
$X = F^{-1}(U_{1}),   U_{1} \sim U(0, 1)$ <br>
$Y = G^{-1}(U_{2}),   U_{2} \sim U(0, 1)$ <br><br>
We say that common random variables are used if $U_{2} = U_{1}$ and antithetic random variables are used if $U_{2} = 1 - U_{1}$. Since both $F^{-1}$ and $G^{-1}$ are nondecreasing functions, in using common random variables, we clearly have <br>
$Cov(F^{-1}(U), G^{-1}(U)) \ge 0$ <br>
for $U \sim U(0, 1)$. Consequently, variance reduction is achieved, in the sense that the estimator $F^{-1}(U) - G^{-1}(U)$ has a smaller variance than the crude Monte Marclo (CMC) estimator X -Y, where X and Y are independent, with cdfs F and G, respectively. 

One of the main application of antithetic random variables is to estimate $l = E[H(X)]$, <br>
where $X \sim F$ is a random vector with independent components and the sample performance function, H(x), is monotonic in each component of x. 

An unbiased estimator of  $l = E[H(X)]$ is the CMC estimator, given by <br><br>
$\hat{l} = \frac{1}{N}\sum_{k=1}^{N}H(X_{k})$, <br><br>
where $X_{1}, .., X_{N}$ is an iid sample from (multidimensional) cdf F. And alternative unbiased estimator of $l$, for even N, is <br><br>
$\hat{l}^{(a)} = \frac{1}{N}\sum_{k=1}^{N/2}{H(X_{k}) + H(X_{k}^{(a)})}$

Variances can be computed as follows: <br><br>
$Var(\hat{l}) = Var(\frac{1}{N}\sum_{k=1}^{N}H(X_{k})) = \frac{1}{N}Var(H(X_{k}))$ <br>

$Var(\hat{l^{(a)}})= Var(\frac{1}{N}\sum_{k=1}^{N/2}{H(X_{k}) + H(X_{k}^{(a)})})$ = $\frac{1}{N^{2}}\frac{N}{2}[Var(H(X_{k})) + Var(H(X_{k}^{(a)})) + Cov(X_{k}, X_{k}^{(a)})]$ = $\frac{1}{2N}[Var(H(X_{k})) + Var(H(X_{k}^{(a)})) + Cov(X_{k}, X_{k}^{(a)})]$ = $\frac{1}{2}[Var(\hat{l}) + Var(\hat{l}) + \frac{Cov(X_{k}, X_{k}^{(a)})}{N}]$ = $Var(\hat{l}) + \frac{Cov(X_{k}, X_{k}^{(a)})}{N}$

From computation point of view if hs is a generated sample of length N in the first case and N/2 in the second case, then: <br>
$Var(\hat{l})$ = np.var(hs) / N <br>
$Var(\hat{l^{(a)}})$ = np.var(hs) / (2N)

#### Problem 5.1 
Consider the integral $\int_{a}^{b}H(x)dx = (b - a)E[H(X)]$, with $X \sim U(a, b)$. Let $X_{1}, .., X_{N}$ be a random sample from $U(a, b)$. <br><br> Consider the estimators $\hat{l} = \frac{1}{N}\sum_{i=1}^{N}H(X_{i})$ and $\hat{l}_{1} = \frac{1}{2N}\sum_{i=1}^{N}H(X_{i}) + H(b + a - X_{i}).$ Prove that if H(x) is monotonic in x, then <br><br>
$Var(\hat{l}_{1}) \le \frac{1}{2}Var(\hat{l})$. <br><br>
In other words, using antithetic random variables is more accurate than using CMC.

#### Problem 5.2 
Estimate the expected length of the shortest path for the bridge network in example 5.1. Use both CMC and the antithetic estimator. For both cases, take a sample size of N = 100,000. Suppose that the lengths of the links $X_{1}, ..., X_{5}$ are exponentially distributed, with means 1, 1, 0.5, 2, 1.5. Compare the results.

#### Solution

In [26]:
N = 100000
MEANS = [1, 1, 0.5, 2, 1.5]
def h(X):
    return np.minimum(
        np.minimum(
            np.minimum(
                X[0] + X[3],
                X[0] + X[2] + X[4]
            ),
            X[1] + X[2] + X[3]
        ), 
        X[1] + X[4]
    )

def inverse_cfd_exp(lambda_, u):
    return -lambda_ * np.log(u)

In [27]:
# crude monte carlo
X = np.zeros((5, N))
for i in range(5):
    X[i] = np.random.exponential(MEANS[i], size=N)
crude_hs = h(X)

In [28]:
# antithetic variables
X = np.zeros((5, int(N / 2)))
X_a = np.zeros((5, int(N / 2)))
for i in range(5):
    U = np.random.uniform(0, 1, size=int(N / 2))
    X[i] = inverse_cfd_exp(MEANS[i], U)
    X_a[i] = inverse_cfd_exp(MEANS[i], 1 - U)

antithetic_hs = h(X) + h(X_a)

In [29]:
len(crude_hs), len(antithetic_hs)

(100000, 50000)

In [30]:
print('crude Monte carlo estimate: {}'.format(np.sum(crude_hs) / N))
print('antithetic random variable estimate: {}'.format(np.sum(antithetic_hs) / N))

crude Monte carlo estimate: 1.4934812999306835
antithetic random variable estimate: 1.5002795607177157


In [32]:
print('crude Monte carlo variance: {}'.format(np.var(crude_hs) / N))
print('antithetic random variable variance: {}'.format(np.var(antithetic_hs) / (2 * N)))

crude Monte carlo variance: 1.0418068551849287e-05
antithetic random variable variance: 5.412574410584615e-06


#### Control variables
Let X be one-dimensional random variable. Let X be an unbiased estimator of $\mu$ (i.e. E[X] = $\mu$), to be obtained from the simulation run. A random variable C is called control variable for X if it is correlated with X and its expectation, r, is known (E[C] = r). The control variable C is used to contruct an unbiased estimator of $\mu$ with a variance smaller than that of X. This estimator, <br><br>
$X_{\alpha} = X - \alpha(C - r)$

The variance of $X_{\alpha}$ is given by <br><br>
$Var(X_{\alpha}) = Var(X) - 2\alpha Cov(X, C) + \alpha^{2}Var(C)$ <br><br>
Consequently, the value $\alpha^{*}$ that minimizes $Var(X_{\alpha})$ is <br><br>
$\alpha^{*} = \frac{Cov(X, C)}{Var(C)}$ <br>

Typically, $\alpha^{*}$ is estimated from the corresponding sample covariance and variance. <br>
Using $\alpha^{*}$, we can write the minimal variance as <br><br>
$Var(X_{\alpha^{*}}) = (1 - \rho_{XC}^{2})Var(X)$, <br>
where $\rho_{XC}$ denotes the correlation coefficient of X and C. The larger $|\rho_{XC}$ is, the greater is the variance reduction. 

#### Problem 5.5
Run the stochastic shortest path problem in Example 5.4 and estimate the performance $l = E[H(X)]$ from 100000 independent replications, using the given $(C_{1}, C_{2}, C_{3}, C_{4})$. Compare the results obtained in problem 5.2.

In [51]:
MEANS

[1, 1, 0.5, 2, 1.5]

if we take $C = X_{1} + X_{4}$, $E[C] = r = 1 + 2 = 3$

In [52]:
N = 100000
r = 3
X = np.zeros((5, N))
for i in range(5):
    X[i] = np.random.exponential(MEANS[i], size=N)
C = X[0] + X[3]

$H_{\alpha}(X) = H(X) - \alpha^{*}(C - r)$

In [53]:
hs = h(X)

In [54]:
# let's first estimate alpha
alpha = np.cov(hs, C)[0, 0] / np.var(C)
alpha

0.21256756632001

In [55]:
hs_alpha = hs - alpha * (C - r)

In [56]:
print('control variable method: estimate = {}'.format(np.mean(hs_alpha)))

control variable method: estimate = 1.5015789577660938


In [57]:
print('control variable method: variance = {}'.format(np.var(hs_alpha) / N))

control variable method: variance = 8.323391925628314e-06


#### Conditional Monte Carlo

## Importance sampling

Let H be a sample performance function and f probability distribution <br>
Very often we are interested in the following quantity: <br> <br>
$l = E_{f}[H(X)] = \int H(x)f(x)dx$


Let g be another probability distribution, which satsfies <br>
$g(x) = 0 \rightarrow H(x)f(x) = 0$

$l = E_{f}[H(X)] = \int H(x)f(x)dx = \int H(x)\frac{f(x)}{g(x)}g(x)dx = E_{g}[H(X)\frac{f(X)}{g(X)}]$

We can distinguish function $W(x) = \frac{f(x)}{g(x)}$ called likelihood ratio.

If $X_{1}, .., X_{N}$ is a random sample from g, that is $X_{1}, .., X_{N}$ are iid random vectors with density g, then <br><br>
$\hat{l} = \frac{1}{N}\sum_{k=1}^{N}H(X_{k})\frac{f(X_{k})}{g(X_{k})}$


#### Variance Minimization Method

We want to keep the variance of the estimator low. $\hat{l}$ is unbiased the estimator of $l$. Let's minimize variance of it, 
by choosing the best distribution g. 

$min_{g}$ $Var_{g}(H(X)\frac{f(X)}{g(X)})$

The solution of it: <br><br>
$g^{*}(x) = \frac{|H(x)|f(x)}{\int|H(x)|f(x)dx}$, if $H(x) \ge 0$ <br><br>
$g^{*}(x) = \frac{H(x)f(x)}{l}$

Let's compute the variance of the optimal distribution: <br><br>
$Var_{g^{*}}(\hat{l}) = Var_{g^{*}}(\frac{1}{N}\sum_{k=1}^{N}H(X_{k})\frac{f(X_{k})}{g^{*}(X_{k})}) = \frac{1}{N}Var_{g^{*}}(H(X_{1})\frac{f(X_{1})}{g^{*}(X_{1})}) = \frac{1}{N}Var_{g^{*}}(l) = 0$, <br> where the last quantity is just a variance of a number. <br><br>
The following approach has only some theoretical value, because in order to get an optimal g we need $l$, which we aspire to eventually estimate. <br>
Minimization very often is done within some distribution family <br><br>
Let $f(\cdot) = f(\cdot; u)$, then we focus on g that comes from the same family. We can formalize: <br><br>
$min_{v}$ $Var_{v} (H(X) \frac{f(X; u)}{f(X; v)})$ <br><br>
$Var_{v} (H(X) \frac{f(X; u)}{f(X; v)}) = E_{v}[(H(X) \frac{f(X; u)}{f(X; v)})^{2}] - E_{v}[(H(X) \frac{f(X; u)}{f(X; v)}]^{2}$ = $E_{v}[(H(X) \frac{f(X; u)}{f(X; v)})^{2}] - l^{2}$.
So equivalently
<br><br>
$V(v) = min_{v}$ $E_{v}[(H(X) \frac{f(X; u)}{f(X; v)})^{2}]$ = $E_{v}[(H(X)W(X; u, v))^{2}]$ = $E_{u}[H(X)^{2}W(X; u, v)]$
The above is mostly theoretical derivation. We can try to find the optimal parameter $v$ through sample. Suppose we have $X_{1}, .., X_{N}$ samples from $f(x;u)$, then <br><br>
$\hat{V}(v) = \frac{1}{N}\sum_{k=1}^{N}[H^{2}(X_{k})W(X_{k};u, v)]$

Assuming we can interchange operators, i.e. <br><br>
$\nabla E_{u}[H^{2}(X)W(X; u, v)] = E_{u}[H^{2}(X)\nabla W(X; u, v)]$ <br><br>
Let's take the derivative <br><br>
$\frac{1}{N}\sum_{k=1}^{N}[H^{2}(X_{k})\nabla W(X_{k};u, v)]$ = 0, where <br><br>
$\nabla W(X_{k};u, v) = \nabla \frac{f(X;u)}{f(X;v)} = [\nabla ln f(X;v)] W(X; u, v)$

#### Cross-Entropy Method

Kullback-Leibler distance: <br><br>
$D(g, h) = E_{g}[ln\frac{g(X)}{h(X)}] = \int g(x) ln(\frac{g(x)}{h(x)})dx = \int g(x) ln(g(x))dx - \int g(x)ln(h(x))dx$.

We want h that is close to optimal (but theorethical) $g^{*}$. Let's search in some family, h = $f(\cdot; v)$. The first term  in the KL distance doesn't depend on h, so we can focus on the second term. Moreover, minimization of KL is equivalent to maximization of the second term. So <br>
$\int g^{*}(x) ln(f(x; v)) dx = \int \frac{H(x)f(x; u)}{l} ln(f(x; v)) dx \propto \int H(x)f(x; u) ln(f(x; v)) dx$ = $E_{u}[H(X)ln f(X; v)]$. Typically, <br><br>
$\nabla E_{u}[H(X)ln f(X; v)] = E_{u}[H(X)\nabla ln f(X; v)]$, so we can try to find v such that <br><br>
$\frac{1}{N}\sum_{k=1}^{N}H(X_{k})\nabla f(X_{k}; v)$ = 0 <br><br>
Let w be an arbitrary reference parameter, we can rewrite <br><br>
$E_{u}[H(X)ln f(X; v)] = \int H(x)f(x;u)ln(f(x;v))\frac{f(x;w)}{f(x;w)}dx$ = $ \int H(x)W(x;u, w)ln(f(x;v))f(x;w)dx$ = $E_{w}[H(X)W(X; u, w)ln(f(X;v)]$ <br><br>
We can estimate $v^{*}$ as the solution of the stochastic program: <br><br>
$\frac{1}{N}\sum_{k=1}^{N}H(X_{k})W(X_{k};u,w)\nabla ln(f(X_{k};v)) = 0$, <br><br>
where $X_{1}, .., X_{N}$ is a random sample from $f(\cdot; w)$. In the next iteration found $v^{*}$ can play the role of w. 